<a href="https://colab.research.google.com/github/kiyomi0917/0417pdsnd_github/blob/main/Ono%EF%BC%BFDA_PJ0623.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install Janome==0.3.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from janome.tokenizer import Tokenizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

# Load the Excel file
file_path = '/content/drive/MyDrive/dataset_0619.xlsx'
xl = pd.ExcelFile(file_path)

In [37]:
# Load the desired sheet into a DataFrame
df1 = xl.parse('0619データ')

# Print the DataFrame
#print(df1)


In [ ]:
# テキストを分割する関数_すべての単語を使う
t=Tokenizer()
def tokenize1(text):
    tokens = t.tokenize(text)
    noun = []
    for token in tokens:
        noun.append(token.surface)
    return noun

# 単語の抽出
def tokenize2(text):
    tokens = t.tokenize(text)
    noun = []
    for token in tokens:
        # 「名詞」「動詞」「形容詞」「形容動詞」を取り出してください
        if token.part_of_speech.split(',')[0] in ['名詞', '動詞', '形容詞', '形容動詞']:
            noun.append(token.surface)
    return noun


In [ ]:
docs = df1["概要"]
labels = df1["ハラスメント判定"].astype(int)  # Convert labels to integer type
print(labels)

0      0
1      0
2      0
3      0
4      0
      ..
108    1
109    1
110    1
111    1
112    1
Name: ハラスメント判定, Length: 113, dtype: int64


In [ ]:
train_data = docs
train_labels =labels
train_labels = train_labels.astype(int)  # Convert labels to integer type

In [ ]:

#from sklearn.metrics import accuracy_score
file_path = '/content/drive/MyDrive/dataset_0619.xlsx'
xl = pd.ExcelFile(file_path)

# Load a sheet into a DataFrame by name
df2 = xl.parse('0620データ')

# テストデータの概要とラベルを取得
test_data = df2["概要"]
test_labels = df2["ハラスメント判定"]

# テキストデータをベクトル化するためのVectorizerを定義
vectorizer = TfidfVectorizer(tokenizer=tokenize2)

# ベクトル化に使用するトークン化関数（tokenize1）を指定してVectorizerを初期化
vectorizer = TfidfVectorizer(tokenizer=tokenize2)

# 学習データをベクトル化
train_matrix = vectorizer.fit_transform(train_data)

# モデルを設定
model = RandomForestClassifier()

# パラメータでモデルを再学習
model.fit(train_matrix, train_labels)

# 訓練データの予測
train_predictions = model.predict(train_matrix)

# 訓練データの正解率を計算して表示
train_accuracy = accuracy_score(train_labels, train_predictions)
print("訓練データの正解率:", train_accuracy)

# テストデータを変換
test_matrix = vectorizer.transform(test_data)

# テストデータの予測
test_predictions = model.predict(test_matrix)

# テストデータの正解率を計算して表示
test_accuracy = accuracy_score(test_labels, test_predictions)
print("テストデータの正解率:", test_accuracy)


In [ ]:
# ハイパーパラメーターの値の候補を設定
model_param_set_grid = {"n_estimators": [50, 100, 200],
                        "max_depth": [None, 10, 20, 30],
                        "random_state": [42]}

# グリッドサーチでハイパーパラメーターを探索
clf = GridSearchCV(model, model_param_set_grid)
clf.fit(train_matrix, train_labels)

# 最適なパラメータとそのときのスコアを表示
print("ハイパーパラメーター:{}".format(clf.best_params_))
# print("ベストスコア:", clf.best_score_)

# 最適なパラメータでモデルを構築
best_model = RandomForestClassifier(**clf.best_params_)

# 最適なパラメータでモデルを再学習
best_model.fit(train_matrix, train_labels)

# 訓練データの予測
train_predictions = best_model.predict(train_matrix)

# 訓練データの正解率を計算して表示
train_accuracy = accuracy_score(train_labels, train_predictions)
print("訓練データの正解率:", train_accuracy)

# テストデータを変換
test_matrix = vectorizer.transform(test_data)

# テストデータの予測
test_predictions = best_model.predict(test_matrix)

# テストデータの正解率を計算して表示
test_accuracy = accuracy_score(test_labels, test_predictions)
print("テストデータの正解率:", test_accuracy)

ハイパーパラメーター:{'max_depth': None, 'n_estimators': 50, 'random_state': 42}
訓練データの正解率: 1.0
テストデータの正解率: 0.7727272727272727


In [ ]:
#from sklearn import svm
import pickle

#学習モデルの保存
with open('model.pickle', mode='wb') as f:
    pickle.dump(best_model,f,protocol=2)
